I had a meeting with Risa the other day and she suggested some of the problem's I've been having were coming from the simulations and not my code. To avoid being distracted, she suggested I basically mock up some results and pull them out. I'm expecting this to be easy because my modeling framework is debugged and coded up. 

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [ ]:
from astropy.io import fits
#dataDir = '/home/sean/Data/BuzzardSims/'
dataDir = '/nfs/slac/g/ki/ki19/des/erykoff/clusters/mocks/Buzzard/buzzard-1.1/des_y5/redmapper_v6.4.7/halos/'
hdulist = fits.open(dataDir+'buzzard-v1.1-y5_run_00340_lambda_chisq.fit')
data = hdulist[1].data

import numpy as np
mass = data['M200']
print 'Rows:\t%d'%mass.shape[0]
redshifts = data['Z']

mass = mass[redshifts<1.7]#for my approximation
redshifts = redshifts[redshifts<1.7]

logMass = np.log10(mass)

The model in the paper Risa sent me is:

$ \log{\lambda}(M) = \log{\lambda_0} + A_{\lambda}\log{\frac{M}{M_{piv}}}+B_{\lambda} \log{\frac{1+z}{1.3}} $

Where $A_{\lambda}$ is allowed to vary with redshift by:

$ A_{\lambda} \equiv a \left(\frac{1+z}{1.3} \right)^b $

The defined/measured values for these params are shown in the table below (with uncertainties surpressed). I will use these to draw a richness sample and then infer them back. 

| Quantity | Value |
|:--- | ----------------------------------------: |
| $M_{piv}$ | $2.35 \times 10^{14} \; M_{\odot} $ | 
| $\log{\lambda_0}$ | 3.141 | 
| $a$ | 0.842 |
| $b$ | -0.03 |
| $ B_{\lambda} $ | 0.642 |
| $ \sigma_{\lambda} $ | 0.184 | 

In [ ]:
M_piv = 2.35e14
log_lam0 = 3.141
a, b = 0.842, -0.03
B_lam = 0.642
sigma_lam = 0.184

def _A_lam(a, b, z):
    return a*pow((1+z)/1.3, b)
from scipy.stats import norm
#forward model
def log_lam(log_lam0, a, b, B_lam, sigma_lam, z, M):
    A_lam = _A_lam(a,b,z)
    mu = log_lam0+A_lam*np.log(M/M_piv)+B_lam*np.log((1+z)/1.3)
    return norm.rvs(loc = mu, scale = sigma_lam)

In [ ]:
log_richness = log_lam(log_lam0, a, b, B_lam, sigma_lam, redshifts, mass)

In [ ]:
plt.hist(log_richness)